## 요구사항

### **기능**

1. 상품을 여러 개 담아 주문을 만든다.
2. 주문 총액을 계산한다.
3. 결제를 수행한다. (결제 방식은 여러 개 가능)
4. 결제가 성공하면 알림을 보낸다.
5. 잘못된 상황(음수 금액, 잔액 부족, 상태 오류)은 예외로 막는다.

### **규칙**

- 주문 상태: CREATED → PAID (결제 전/후만 다룸)
- 결제 실패 시 주문 상태는 바뀌면 안 됨.
- 결제와 알림은 교체 가능 해야 함

In [1]:
class Order:
    def __init__(self):
        self.items = []
        self.status = "CREATED"

    def add_item(self, name, price, qty):
        self.items.append((name, price, qty))

    def total(self):
        return sum(price * qty for _, price, qty in self.items)

    def pay_and_notify(self):
        amount = self.total()
        if amount <= 0:
            raise ValueError("총액이 0 이하여서 결제할 수 없습니다.")
        # 결제(하드코딩)
        print(f"[PAY] {amount}원 결제 요청")
        self.status = "PAID"
        # 알림(하드코딩)
        print("[NOTIFY] 결제 완료 알림 전송")

In [ ]:
from abc import ABC, abstractmethod
from typing import override
from enum import Enum
from dataclasses import dataclass

# notifier, payment, order, checkout 모듈은 책임 별로 분리하여 SRP 준수


# notifier.py 
class Notifier(ABC):  # 외부에서 사용하는 클래스(모듈)이기에 강제성 갖추도록, Protocol 대신 ABC 사용. PaymentService의 내부 모듈이라면 Protocol 사용하는 것이 Pythonic(과도한 추상화 X) 하다.  (but, 하나의 메서드 밖에 없는데 ABC 사용하는 것은 Pythonic 하지 않다.)
		@abstractmethod
		def send(self, message: str) -> None:  # 다형성 위해 추상 메서드 선언
				...
				
class KakaoTalkNotifier(Notifier):
		@override  # 적지 않아도 되지만, typing 위해 오버라이딩 표시
		def send(self, message: str) -> None:
				print(message)



# payment.py
class PaymentError(Exception):
		pass


class PaymentService(ABC):
		def __init__(self, balance: int):
				self._balance = balance
		
		@abstractmethod
		def pay(self, amount: int) -> None:
				...

class KakaoBankPaymentService(PaymentService):
		def __init__(self, balance: int, notifier: Notifier):  # 의존성 주입, OCP 준수
				self._notifier = notifier
				
				super().__init__(balance)

		@override
		def pay(self, amount: int) -> None:
				if amount <= 0:
						raise ValueError("amount must be positive")  # Exception은 대처하지 못하는 경우 raise 하는 것이 좋다. 그리고 대처할 수 있는 곳에서 try except 처리하는 것이 좋다. (대처할 수 없는데 중간에서 try except 처리하는 것은 코드를 복잡하게 만든다.)
				
				if self._balance < amount:
						raise PaymentError("Not Enough Balance")
				
				self._balance -= amount
				self._notifier.send(f"[NOTIFY] {amount} 결제 완료")



# order.py
class OrderStatus(str, Enum):  # Enum 통해 str 타입 변수 관리
		CREATED = "CREATED"
		PAID = "PAID"

@dataclass
class Product:  # dataclass 통해 구조체 형성. 검증 기능 많이 필요하지 않다면 pydantic 대신 사용하는 것이 가볍고 빠르다.
		name: str
		price: int  # 만약 int, float 타입 바꿔 낄 수 있어야 한다면, TypeVar, Generic 사용
		
class OrderItem:
		def __init__(self, product: Product, qty: int):
				self._product = product
				self._qty = qty
		
		def total(self) -> int:  # 외부에서 내부 코드 확인 필요없이 해당 인터페이스만 사용하도록 추상화
				return self._product.price * self._qty

class Order:  # status, items는 노출된 Interface(is_paid, set_paid, add_item) 통해서만 접근하도록 캡슐화. 
		def __init__(self):
				self._items = []
				self._status = OrderStatus.CREATED
		
		def is_paid(self) -> bool:
				return self._status == OrderStatus.PAID
		
		def add_item(self, item: OrderItem) -> None:
				self._items.append(item)
				
		def total(self) -> int:
				return sum(item.total() for item in self._items)
				
		def set_paid(self) -> None:				
				self._status = OrderStatus.PAID


class OrderError(Exception):
		pass


class CheckoutService:
		def checkout(self, order: Order, payment: PaymentService) -> None:  # 의존성 주입
				if order.is_paid():
						raise OrderError("order already paid")
				
				amount = order.total()
				
				try:
						payment.pay(amount)
				except (PaymentError, ValueError) as e:
						raise OrderError(f"checkout failed: {e}") from e
				
				else:
						order.set_paid()



# main.py
## customer setting
kakaotalk_notifier = KakaoTalkNotifier()
my_payment = KakaoBankPaymentService(balance=1000, notifier=kakaotalk_notifier)

## market setting
checkout_service = CheckoutService()
apple = Product(name='apple', price=10)
banana = Product(name='banana', price=20)

## customer actions
my_order = Order()

apple_item = OrderItem(apple, 4)
banana_item = OrderItem(banana, 1)

my_order.add_item(apple_item)
my_order.add_item(banana_item)

## market actions
checkout_service.checkout(my_order, my_payment)

[NOTIFY] 60 결제 완료
